In [49]:
import json
from urllib.parse import urlparse, parse_qs

# get the result json file and convert it to geojson to ease the debugging with QGIS
def closest_results_create_geojson(file_path):
    # Open the JSON file and load the data into a dictionary
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Create a new dictionary for the GeoJSON data
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    # Iterate over the keys and values in the original dictionary
    for key, value in data.items():
        query = value['query']
        parsed_url = urlparse(query)

        # Get the query parameters
        query_params = parse_qs(parsed_url.query)
        lat, lon = query_params['point'][0].split(",")
        lat = float(lat)
        lon = float(lon)
        
        if isinstance(value, dict) and value.get('result', {}):
            # create a feature for the input
            input_feature = {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [lon, lat]},
                "properties": {"type": "input", "distance": value['result']['distance']},
            }
        
            output_feature = {
                "type": "Feature",
                "geometry": value['result']['point'],
                "properties": value['result'],
            }
            
            line_feature = {
                "type": "Feature",
                "geometry": {
                "type": "LineString",
                "coordinates": [
                    [lon, lat],
                    value['result']['point']['coordinates']
                ]},
                "properties": value['result']['distance']
            }

            # Add the feature dictionary to the features list
            geojson['features'].append(input_feature)
            geojson['features'].append(output_feature)
            geojson['features'].append(line_feature)

    return geojson

def save_geojson(geojson, file_name):
    with open(file_name, 'w') as f:
        json.dump(geojson, f)

def convert_closest_to_geojson_file(file_path):
    geojson = closest_results_create_geojson(file_path)
    save_geojson(geojson, file_path + ".geojson")

In [50]:
convert_closest_to_geojson_file("Enseignement supérieur/results/results-closest.json")